In [ ]:
import numpy as np
import scipy
import scipy.linalg
import inference

from matplotlib import pyplot as plt
from scipy.stats import ortho_group

In [ ]:
def test_random_projection(levels=[1,3,9,27]):
    """Within an n dimensional space, project an orthogonal set of k vectors (a k-dimensional subspace) into a space of dimension n-k
    where subspaces of size "levels" are shared"""

    nsamples = 100

    n = levels[-1]
    testdim = np.arange(1,levels[-1]+1)
    
    error = np.zeros((testdim.size,nsamples))    
    for i in range(nsamples):
        U = ortho_group.rvs(dim=n)
        V = randomize_vectors(U,levels)
        for (j,k) in enumerate(testdim):
            U2 = U[:,:k]
            V2 = V[:,:k] 
            if k==1:
                continue
            else:
                error[j,i] = calculate_proj_error(U2,V2)
    
    meanerror = np.mean(error,axis=1)
    stderror = np.std(error,axis=1)
    

# OLD VARIANT -- there is a bug somewhere!?!
#     nr_shared_vectors = 9
#     for (j, k) in enumerate(testdim):
#         # create an orthogonal set of random vectors
#         U = ortho_group.rvs(dim=n)
#         U_rotated = U[:,1:nr_shared_vectors]
#         U = U[:, :k]
# #         print(U)
# #         print(U_rotated[:,:nr_shared_vectors] )
#         error = np.zeros(nsamples)
#         if k==1:
#             continue
#         for i in range(nsamples):
#             # create k orthogonal vectors
#             V = ortho_group.rvs(dim=n)
#             V = V[:, :k]
#             U_rotated2 = U_rotated @ ortho_group.rvs(dim=nr_shared_vectors-1) 
#             U_rotated2 = U_rotated2[:,:k]
#             V[:,1:nr_shared_vectors] = U_rotated2[:,1:nr_shared_vectors] 
#             V[:,0]= U[:,0]
#             V = randomize_last_k_vectors(U,k)
#             error[i] = calculate_proj_error(U, V)
#         meanerror[j] = np.mean(error)
#         stderror[j] = np.std(error)

    expected_error = expected_errors_random_projection_new(levels)
    expected_error_as_is = inference.expected_errors_random_projection(levels)
    plt.figure()
    plt.errorbar(testdim, meanerror, stderror,label="empirical")
    plt.errorbar(testdim, expected_error,label="updated calculation")
    plt.errorbar(testdim, expected_error_as_is, label="old calculation / paper")
    plt.legend()
    
def randomize_vectors(U,levels):
    """Given a complete set of basis vectors, keep the subspaces spanned at specified levels intact, but randomize the basis vectors otherwise (by random rotatations)"""
    V = U.copy()
    start_end_pairs = zip(levels[:-1], levels[1:])
    for start, end in start_end_pairs:
        V[:,start:end] = V[:,start:end] @ ortho_group.rvs(dim=end-start)
    return V
                                                 

In [ ]:
def project_orthogonal_to(subspace_basis, vectors_to_project):
    """
    Subspace basis: linearly independent (not necessarily orthogonal or normalized)
    vectors that span the space orthogonal to which we want to project
    vectors_to_project: project these vectors into the orthogonal complement of the
    specified subspace
    """

    if not scipy.sparse.issparse(vectors_to_project):
        V = np.matrix(vectors_to_project)
    else:
        V = vectors_to_project

    orthogonal_proj = V - project_to(subspace_basis, V)

    return orthogonal_proj

def project_to(subspace_basis, vectors_to_project):
    """
    Subspace basis: linearly independent (not necessarily orthogonal or normalized)
    vectors that span the space to which we want to project
    vectors_to_project: project these vectors into the specified subspace
    """

    if not scipy.sparse.issparse(vectors_to_project):
        V = np.matrix(vectors_to_project)
    else:
        V = vectors_to_project

    if not scipy.sparse.issparse(subspace_basis):
        S = np.matrix(subspace_basis)
    else:
        S = subspace_basis

    # compute S*(S^T*S)^{-1}*S'*V
    X1 = S.T * V
    X2 = S.T * S
    projected = S * scipy.linalg.solve(X2, X1)

    return projected

def calculate_proj_error(U, V):
    proj1 = project_orthogonal_to(U, V)
    error = scipy.linalg.norm(proj1)**2

    return error

def expected_errors_random_projection_new(levels):
    """
    Compute vector of expected errors for random projection
        dim_n -- ambient space
        levels of hierarchy [1, ..., dim_n]
    """
    start_end_pairs = zip(levels[:-1], levels[1:])

    expected_error = []
    for i, j in start_end_pairs:
        Ks = np.arange(i, j)
        errors = (Ks - i) * (j - Ks) / (j)
        expected_error = np.hstack([expected_error, errors])
    expected_error = np.hstack([expected_error, 0])
    return expected_error

In [ ]:
test_random_projection()

## Second Test
Now instead of random vectors and sub-space projections we use the eigenvectors from the Laplacian of an aggregated network and create the second set of vectors for the projection based on k-means

In [ ]:
import generation
import cluster
from spectral_operators import UniformRandomWalk
import warnings
warnings.filterwarnings('ignore')

c_bar = 50
n = 3**9
snr = 6
nsamples = 10
max_groups = 27
testdim = np.arange(1,max_groups+1)

dendro = generation.create2paramGHRG(n, snr, c_bar, 3, 3)
true_part = dendro[-1]

error = np.zeros((max_groups,nsamples))


for i in range(nsamples):
    A = dendro.sample_network()
    Eagg, Nagg = true_part.count_links_between_groups(A)
    Aagg = Eagg / Nagg

    L = UniformRandomWalk(Aagg)
    L.find_k_eigenvectors(max_groups, which='LM')

    partition_list = []

    for k in range(1, max_groups+1):
        partition = cluster.find_partition(L.evecs, k, normalization=True)
        partition_list.append(partition)

    for ki, partition in enumerate(partition_list):
        error[ki,i] = partition.calculate_proj_error(L.evecs, 'Fnew')
        
        
meanerror = np.mean(error,axis=1)
stderror = np.std(error,axis=1)
levels=[1,3,9,27]
expected_error = expected_errors_random_projection_new(levels)
expected_error_as_is = inference.expected_errors_random_projection(levels)

plt.figure()
plt.errorbar(testdim, meanerror, stderror,label="empirical kmeans")
plt.errorbar(testdim, expected_error,label="updated calculation")
plt.errorbar(testdim, expected_error_as_is, label="old calculation / paper")
plt.legend()